# CANONICAL Review Workflow

Human-in-the-loop validation for promoting fields to CANONICAL status.

## Promotion Laws

| Tier | Meaning | Promotion Path |
|------|---------|----------------|
| THEORETICAL | Described but unseen | Observation |
| OBSERVED | Seen in recordings | 3+ examples |
| VERIFIED | Verified by agent | Human approval |
| **CANONICAL** | Production-ready | - |

In [ ]:
from _paths import *
import json
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from datetime import datetime

## 1. Load Current Status

Load discovered schemas and identify promotion candidates:

In [ ]:
schemas = load_discovered_schemas()
canonical_spec = load_canonical_spec()

# Categorize by tier
tiers = {'THEORETICAL': [], 'OBSERVED': [], 'VERIFIED': [], 'CANONICAL': []}

for event_name, schema in schemas.items():
    tier = schema.get('tier', 'OBSERVED')
    sample_count = schema.get('sample_count', 0)
    tiers[tier].append({
        'event': event_name,
        'samples': sample_count,
        'fields': len(schema.get('fields', {}))
    })

print("Current Status:")
print("=" * 40)
for tier, events in tiers.items():
    print(f"{tier}: {len(events)} events")

## 2. Promotion Candidates

Events ready for CANONICAL promotion (VERIFIED tier + human approval):

In [ ]:
# Find VERIFIED events ready for promotion
candidates = []
for event_name, schema in schemas.items():
    if schema.get('tier') == 'VERIFIED':
        candidates.append({
            'event': event_name,
            'samples': schema.get('sample_count', 0),
            'fields': schema.get('fields', {}),
            'verified_by': schema.get('verified_by', 'unknown'),
            'verified_at': schema.get('verified_at', 'unknown')
        })

if candidates:
    print(f"Found {len(candidates)} candidates for CANONICAL promotion:\n")
    for c in candidates:
        print(f"  {c['event']}")
        print(f"    Samples: {c['samples']}, Fields: {len(c['fields'])}")
        print(f"    Verified by: {c['verified_by']}")
else:
    print("No VERIFIED events pending promotion.")
    print("\nTo promote to VERIFIED, an event needs:")
    print("  - 3+ observed samples")
    print("  - Agent verification of field types")

## 3. Interactive Review Widget

Review and approve events for CANONICAL promotion:

In [ ]:
# State for tracking approvals
approvals = {}

def create_review_widget(event_data):
    """Create interactive review widget for an event."""
    event_name = event_data['event']
    
    # Header
    header = widgets.HTML(f"<h3>{event_name}</h3>")
    
    # Event details
    details = widgets.HTML(f"""
    <div style='background: #f5f5f5; padding: 10px; margin: 5px 0;'>
        <b>Samples:</b> {event_data['samples']}<br>
        <b>Fields:</b> {len(event_data['fields'])}<br>
        <b>Verified by:</b> {event_data['verified_by']}<br>
        <b>Verified at:</b> {event_data['verified_at']}
    </div>
    """)
    
    # Fields display
    fields_html = "<b>Fields:</b><ul>"
    for field_path, field_info in list(event_data['fields'].items())[:10]:
        field_type = field_info.get('type', 'unknown')
        fields_html += f"<li><code>{field_path}</code>: {field_type}</li>"
    if len(event_data['fields']) > 10:
        fields_html += f"<li>... and {len(event_data['fields']) - 10} more</li>"
    fields_html += "</ul>"
    fields_display = widgets.HTML(fields_html)
    
    # Approval buttons
    approve_btn = widgets.Button(description="APPROVE", button_style='success')
    reject_btn = widgets.Button(description="REJECT", button_style='danger')
    skip_btn = widgets.Button(description="Skip", button_style='')
    
    # Notes field
    notes = widgets.Textarea(
        placeholder='Optional notes for this decision...',
        layout=widgets.Layout(width='100%', height='60px')
    )
    
    # Status output
    status = widgets.Output()
    
    def on_approve(b):
        approvals[event_name] = {
            'decision': 'APPROVED',
            'notes': notes.value,
            'timestamp': datetime.now().isoformat()
        }
        with status:
            clear_output()
            print(f"APPROVED {event_name} for CANONICAL promotion")
    
    def on_reject(b):
        approvals[event_name] = {
            'decision': 'REJECTED',
            'notes': notes.value,
            'timestamp': datetime.now().isoformat()
        }
        with status:
            clear_output()
            print(f"REJECTED {event_name} - remains VERIFIED")
    
    def on_skip(b):
        approvals[event_name] = {'decision': 'SKIPPED'}
        with status:
            clear_output()
            print(f"Skipped {event_name}")
    
    approve_btn.on_click(on_approve)
    reject_btn.on_click(on_reject)
    skip_btn.on_click(on_skip)
    
    buttons = widgets.HBox([approve_btn, reject_btn, skip_btn])
    
    return widgets.VBox([
        header, details, fields_display, notes, buttons, status
    ])

# Display review widgets
if candidates:
    print("Review each event and click APPROVE, REJECT, or Skip:\n")
    for candidate in candidates:
        display(create_review_widget(candidate))
        display(widgets.HTML("<hr>"))
else:
    print("No candidates to review.")
    print("\nTo test the widget, run the simulation cell below.")

## 4. Apply Approvals

Write approved promotions to the canonical spec:

In [ ]:
def apply_approvals():
    """Apply approved promotions to the schema files."""
    approved = [k for k, v in approvals.items() if v.get('decision') == 'APPROVED']
    rejected = [k for k, v in approvals.items() if v.get('decision') == 'REJECTED']
    
    print("Approval Summary:")
    print("=" * 40)
    print(f"Approved: {len(approved)}")
    print(f"Rejected: {len(rejected)}")
    print(f"Skipped:  {len(approvals) - len(approved) - len(rejected)}")
    
    if approved:
        print("\nPromoted to CANONICAL:")
        for event_name in approved:
            print(f"  - {event_name}")
        
        # TODO: Update discovered_schemas.json with new tiers
        # TODO: Append to WEBSOCKET_EVENTS_SPEC.md
        print("\n[DRY RUN] - Actual file updates not yet implemented.")
        print("Files that would be updated:")
        print(f"  - {KNOWLEDGE_PATH}/generated/discovered_schemas.json")
        print(f"  - {KNOWLEDGE_PATH}/WEBSOCKET_EVENTS_SPEC.md")
    
    return approved, rejected

# Run this after reviewing all candidates
if approvals:
    approved, rejected = apply_approvals()
else:
    print("No decisions recorded yet.")
    print("Use the review widgets above to approve/reject events.")

## 5. Simulation Mode

Test the review workflow with simulated data:

In [ ]:
# Simulate candidates for testing the widget
simulated_candidates = [
    {
        'event': 'gameStateUpdate',
        'samples': 15000,
        'fields': {
            'data.price': {'type': 'float'},
            'data.volume': {'type': 'int'},
            'data.timestamp': {'type': 'int'}
        },
        'verified_by': 'rugs-expert-agent',
        'verified_at': '2025-12-22T10:00:00'
    },
    {
        'event': 'playerJoined',
        'samples': 500,
        'fields': {
            'data.playerId': {'type': 'string'},
            'data.walletAddress': {'type': 'string'}
        },
        'verified_by': 'rugs-expert-agent',
        'verified_at': '2025-12-22T10:00:00'
    }
]

print("Simulated Review (for testing widget functionality):\n")
for candidate in simulated_candidates:
    display(create_review_widget(candidate))
    display(widgets.HTML("<hr>"))

## Audit Log

All promotion decisions are logged for compliance:

In [ ]:
def save_audit_log():
    """Save approval decisions to audit log."""
    audit_path = KNOWLEDGE_PATH / "generated" / "promotion_audit.json"
    
    existing = []
    if audit_path.exists():
        with open(audit_path) as f:
            existing = json.load(f)
    
    # Add new entries
    for event_name, decision in approvals.items():
        if decision.get('decision') in ('APPROVED', 'REJECTED'):
            existing.append({
                'event': event_name,
                **decision,
                'session': datetime.now().isoformat()
            })
    
    # Save
    audit_path.parent.mkdir(parents=True, exist_ok=True)
    with open(audit_path, 'w') as f:
        json.dump(existing, f, indent=2)
    
    print(f"Audit log saved to: {audit_path}")
    print(f"Total entries: {len(existing)}")

# Uncomment to save:
# save_audit_log()

## Next Steps

- **03_coverage_dashboard.ipynb** - Visualize documentation coverage
- **04_rl_bot_analysis.ipynb** - Analyze RL model performance